# Export Data

Generalized export class to transfer data from scraper to application processing pipeline

1. Query Data (->Read_Query)
2. Transform - pipe dependent, base class inheritance with general methods
3. Save & Upload

In [57]:
import pandas as pd
from awstools import write_data, generate_job
from read_query import dump_businesses


### Base Class ###

class Exporter():
    def __init__(self):
        pass
    
    def fetch(self):
        NotImplemented
    
    def transform(self):
        NotImplemented
    
    def save_s3(self, tdata: pd.DataFrame, savepath):
        write_data(
            savepath = savepath,
            filetype = 'json',
            dry_run = False,
        )
        self.generate_job()
        
    def export(self):
        pass
        
    def generate_job(self, savepath, job_type, tablename, **kwargs):
        generate_job(
            obectpath = savepath,
            job_type = job_type,
            tablename = tablename,
            **kwargs
        )
    
    def fetch_transform(self):
        return self.transform(
                data = self.fetch()
            )
        
    
    
### Derived Classes ###

class BusinessExport(Exporter):
    def __init__(self):
        super().__init__()
        
    def fetch(self, columns=[
                            'business_id', 'name', 'address', 'city', 'state',
                            'latitude', 'longitude', 'postal_code', 'review_count', 'stars', 
                            'is_open', 'categories']):
        self.data_ = dump_businesses()
        return pd.DataFrame(self.data_, columns=columns)
    
    def transform(self, data=None):
        if data is None and self.data_:
            self.transformed_data = self.data_
            return self.data_
        elif data is not None:
            self.transformed_data = data
            return data # Null transform returns data
        else:
            raise ValueError
        
    def export(self, filename, data=None):
        if data is not None:
            data_to_export = data
        else:
            data_to_export = self.transformed_data
        data_to_export.to_json(orient='records',path_or_buf=filename) 
        
        

In [58]:
business_exporter = BusinessExport()

In [59]:
bdata = business_exporter.fetch_transform()
bdata.head()

,business_id,name,address,city,state,latitude,longitude,postal_code,review_count,stars,is_open,categories
0,LnSPs3KafwvvsPjSZZIpIw,Nine Minute Oil & Lube,3839 El Camino Real,Palo Alto,CA,37.417688,-122.131003,94306,1237,5.0,1,"autorepair,oilchange"
1,mD1TFPyxCqAnQkxOyEbMJQ,Super Auto Glass,61 Airport BlvdSte A,South San Francisco,CA,37.650768,-122.409095,94080,575,5.0,1,"autoglass,windshieldinstallrepair"
2,ZBGyB5JEdXWq52yvp78kxw,Putnam Subaru,85 California Dr,Burlingame,CA,37.577580,-122.341340,94010,1038,4.5,1,"car_dealers,autorepair,autopartssupplies"
3,Ut12obILF409DeIMZYozEg,JP Auto Body Shop,112 S Linden AveNone,South San Francisco,CA,37.646030,-122.413810,94080,375,5.0,1,bodyshops
4,WjgkyKvlJ9vy3_juKKdMFw,Golden State Smog Center,1150 Old County Rd,Belmont,CA,37.519844,-122.274261,94002,493,5.0,1,smog_check_stations


In [60]:
business_exporter.export(filename='test_biz.json')

## Reviews Exporter

New subclass for exporting reviews.  

In [3]:
from exports import ReviewsExport

review_exporter = ReviewsExport()

In [4]:
rdata = review_exporter.fetch_transform()
rdata

,review_id,date,stars,text,business_id,user_id
0,mpmdM7CeDRsXGpXXzo56zA,2018-03-15 19:35:24,5.0,"Sean, the assistant manager, rocked. Took the ...",-9ZGZEnaxbLwwM91fVA7WA,09JZistwSnDS8Sw6GE_3dw
1,M6Nq4Cji-ZYuDX1lbPSflg,2019-12-13 19:05:30,1.0,I think we are done with this place. Whatever ...,-9ZGZEnaxbLwwM91fVA7WA,U8LLCHCFqvZBzUtq-Loxhg
2,FnlWc_Pes2XcY_MNGsr0xg,2019-05-03 13:45:47,3.0,I ordered Pizza from here just because they we...,-9ZGZEnaxbLwwM91fVA7WA,3wRJgNtZSVA02LVTV-Q95A
3,Vxij6_Xt8kLHFOTkakYLNg,2019-12-08 18:27:04,5.0,I want to say if you are looking for awesome c...,-9Ij2nWs6I1AQ9al1yjbug,7Rr03lZFyYvYRAaqzmlQlg
4,DXdyHbmR02DfyvLFc2Rrew,2018-05-13 11:18:40,5.0,I have definitely been converted into a Daniel...,-9Ij2nWs6I1AQ9al1yjbug,BA3rFDdHZQ1W2YX68qLjKw
5,LP9P2pVf2jl2ZUsyxlorrg,2019-08-13 06:50:41,1.0,"If I could give ""zero"" stars... I would. I mes...",-9Ij2nWs6I1AQ9al1yjbug,oMHhtj_EZ30IlyRwvN-DhA
6,bW-sudM0gAAQQjkQTqVzAw,2019-07-28 19:28:13,5.0,Very friendly and accommodating service!!! \n\...,-8YkWBXs1ezZ5zGz5YxLhQ,egMYdMR_vl9kGJucOvK9zA
7,HHciSwDbDFbm0mr8NZa9zA,2019-09-11 19:55:42,5.0,These guys are the best! I had to get my winds...,-8YkWBXs1ezZ5zGz5YxLhQ,LRjl5qb6OGQi599MNhQ_xA
8,gnPhKuiMgGiyeoM99Fbaeg,2019-08-09 20:25:52,5.0,After driving around with a few rock chips and...,-8YkWBXs1ezZ5zGz5YxLhQ,2hGZ3LHq8okcTBMCoCpXjg
9,5RSuqn6rsf-GyUF_omeGHg,2019-10-17 09:01:21,5.0,Excellent service and attention to detail. Had...,-8BFkITfrl7VHVz6gqxg3g,QtLbx3pbNGqrWKkaH7bf1g


In [5]:
reviews_exporter.export(filename='test_reviews.json')

NameError: name 'reviews_exporter' is not defined